<a href="https://colab.research.google.com/github/zeiosis/ffnet-summary-prediction/blob/main/dangerzone/colab/ff_analysis_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#maybe define custom loss function? idk

#Library imports and setup

In [ ]:
!pip install -U transformers==4.15.0 --quiet
!pip install -U sentence-transformers==2.2.0 --quiet
!pip install -U transformers[sentencepiece] --quiet
!pip install datasets --quiet

import transformers
from transformers import pipeline, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, AutoModelForSequenceClassification, EvalPrediction
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np
from huggingface_hub import notebook_login
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import torch

     |████████████████████████████████| 3.4 MB 9.8 MB/s 
     |████████████████████████████████| 880 kB 51.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 79 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 19.4 MB/s 
     |████████████████████████████████| 346 kB 7.8 MB/s 
     |████████████████████████████████| 212 kB 63.9 MB/s 
     |████████████████████████████████| 140 kB 68.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.4 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 66.7 MB/s 
     |████████████████████████████████| 271 kB 72.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 68.8 MB/s 
     |████████████████████████████████| 112 kB 71.3 MB/s 
ERROR: pip's dependency 

In [ ]:
print(f"Is CUDA supported by this system? \
      {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:\
      {torch.cuda.current_device()}")
        
print(f"Name of current CUDA device:\
      {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system?       True
CUDA version: 11.3
ID of current CUDA device:      0
Name of current CUDA device:      Tesla T4


#Data processing

In [ ]:
raw_datasets = pd.read_csv("ff_train_new.csv") #training data, in df-to-csv'ed form. original df should be one-hot.
raw_datasets = raw_datasets.rename(columns={'Summary': 'text', 'Unnamed: 0': 'ID'}) #i'm too lazy to figure out how to change the code

raw_datasets = Dataset.from_dict(raw_datasets)

# 90% train, 10% test + validation
ttv_ds = raw_datasets.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid = ttv_ds['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
raw_datasets = DatasetDict({
    'train': ttv_ds['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


checkpoint = 'zdreiosis/ff_analysis_2' ###maybe I have to change this bit? which one should I use: test. previously "zdreiosis/bert-finetuned-sem_eval-english" (bc last one no?)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) 

Downloading:   0%|          | 0.00/321 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
labels = [label for label in raw_datasets['train'].features.keys() if label not in ['ID', 'text']] #finds labels ('genres') in columns
id2label = {idx:label for idx, label in enumerate(labels)} 
label2id = {label:idx for idx, label in enumerate(labels)} #creates dicts for id/label (&v/v) mappings

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") ###why do we have two tokenizer definitions? Cell 1 already defines this as from_pretrained(checkpoint); is this different? shoul

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label] #definitely not my code sadface

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset = raw_datasets.map(preprocess_data, batched=True, remove_columns=raw_datasets['train'].column_names)
encoded_dataset.set_format("torch")

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#HF login (must do manually)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


#Model setup

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("zdreiosis/ff_analysis_3",  ###goddammit not again
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
batch_size = 32
metric_name = "f1"

In [ ]:
args = TrainingArguments(
    f"ff_analysis_4",
    evaluation_strategy = "steps", #epoch
    #save_strategy = "no", #epoch
    eval_steps=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

In [ ]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result #pretty sure I have to change this somehow to make a custom loss fx? otherwise reported will be misleading, but does this impact the training?

In [ ]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))

#Training

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/zdreiosis/ff_analysis_3 into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.58k/418M [00:00<?, ?B/s]

Download file runs/Jun08_09-49-14_e693487630e5/1654681791.219407/events.out.tfevents.1654681791.e693487630e5.7…

Download file runs/Jun08_09-49-14_e693487630e5/events.out.tfevents.1654682963.e693487630e5.71.2: 100%|########…

Download file training_args.bin: 100%|##########| 2.86k/2.86k [00:00<?, ?B/s]

...

Download file runs/Jun08_09-49-14_e693487630e5/events.out.tfevents.1654681791.e693487630e5.71.0:  22%|##1     …

Clean file runs/Jun08_09-49-14_e693487630e5/events.out.tfevents.1654682963.e693487630e5.71.2: 100%|##########|…

Clean file training_args.bin:  35%|###4      | 1.00k/2.86k [00:00<?, ?B/s]

Clean file runs/Jun08_09-49-14_e693487630e5/events.out.tfevents.1654681791.e693487630e5.71.0:   6%|6         |…

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1558
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1470


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
50,No log,0.013765,1.000000,1.000000,1.000000
100,No log,0.013231,0.996633,0.996644,0.988506
150,No log,0.009746,1.000000,1.000000,1.000000
200,No log,0.009496,0.996633,0.996644,0.988506
250,No log,0.009575,1.000000,1.000000,1.000000
300,No log,0.007913,1.000000,1.000000,1.000000
350,No log,0.006993,1.000000,1.000000,1.000000
400,No log,0.006894,1.000000,1.000000,1.000000
450,No log,0.006491,1.000000,1.000000,1.000000
500,0.012000,0.006034,1.000000,1.000000,1.000000


***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
***** Running Evaluation *****
  Num examples = 87
  Batch size = 32
Saving model checkpoint to ff_analysis_3/checkpoint-500
Configuration saved in ff_analysis_3/checkpoint-500/config.json
Model weights saved in ff_analysis_3/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ff_analysis_3/checkpoint-500/tokenizer_config.json
Special tokens file saved in ff_analys

TrainOutput(global_step=1470, training_loss=0.0066953362250814635, metrics={'train_runtime': 1093.709, 'train_samples_per_second': 42.735, 'train_steps_per_second': 1.344, 'total_flos': 3074618307502080.0, 'train_loss': 0.0066953362250814635, 'epoch': 30.0})

#Results

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 87
  Batch size = 32


{'epoch': 30.0,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_loss': 0.006033584475517273,
 'eval_roc_auc': 1.0,
 'eval_runtime': 0.7358,
 'eval_samples_per_second': 118.241,
 'eval_steps_per_second': 4.077}

In [ ]:
trainer.push_to_hub(tags="6th", commit_message="30epx/n1")   #uh huh chage these awful tags pls

Saving model checkpoint to ff_analysis_3
Configuration saved in ff_analysis_3/config.json
Model weights saved in ff_analysis_3/pytorch_model.bin
tokenizer config file saved in ff_analysis_3/tokenizer_config.json
Special tokens file saved in ff_analysis_3/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Jun08_10-23-08_e13d3a3c8876/events.out.tfevents.1654683966.e13d3a3c8876.71.0:  21%|##        …

Upload file runs/Jun08_10-23-08_e13d3a3c8876/events.out.tfevents.1654685264.e13d3a3c8876.71.2: 100%|##########…

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/zdreiosis/ff_analysis_3
   4466abd..65500c3  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 1.0}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 1.0}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/zdreiosis/ff_analysis_3
   65500c3..c088c1e  main -> main



'https://huggingface.co/zdreiosis/ff_analysis_3/commit/65500c3d0bad8720f61fc0dd7bfa868259216f7a'